In [162]:
import pandas as pd

In [163]:
stats=pd.read_csv(r"datasets/player_mvp_stats_adv.csv")
del stats['Unnamed: 0']

In [164]:
stats

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,Doc Rivers,PG,29,ATL,79,79,32.7,5.6,12.9,0.435,...,9.9,20.1,4.1,2.2,6.3,0.116,1.0,1.0,1.9,2.6
1,Dominique Wilkins,SF,31,ATL,81,81,38.0,9.5,20.2,0.470,...,9.6,28.1,8.6,2.8,11.4,0.177,5.4,-0.5,4.9,5.4
2,Duane Ferrell,SF,25,ATL,78,2,14.9,2.2,4.6,0.489,...,15.5,17.8,1.3,0.6,1.9,0.079,-1.3,-1.1,-2.4,-0.1
3,Gary Leonard,C,23,ATL,4,0,2.3,0.0,0.0,0.000,...,0.0,8.1,0.0,0.0,0.0,0.114,-6.3,1.1,-5.2,0.0
4,John Battle,SG,28,ATL,79,2,23.6,5.0,10.9,0.461,...,10.1,24.7,3.4,0.6,4.0,0.102,0.7,-1.6,-0.9,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14087,Raul Neto,PG,28,WAS,64,22,21.9,3.3,7.0,0.468,...,9.9,15.8,1.5,1.3,2.8,0.097,-1.4,0.9,-0.6,0.5
14088,Robin Lopez,C,32,WAS,71,9,19.1,3.8,6.0,0.633,...,13.4,17.1,2.8,0.9,3.7,0.131,0.2,-1.2,-1.0,0.4
14089,Rui Hachimura,PF,22,WAS,57,57,31.5,5.4,11.4,0.478,...,8.6,18.1,0.7,1.4,2.1,0.056,-2.2,-1.0,-3.1,-0.5
14090,Russell Westbrook,PG,32,WAS,65,65,36.4,8.4,19.0,0.439,...,18.0,30.2,0.5,3.2,3.7,0.075,2.6,1.1,3.7,3.4


In [165]:
pd.set_option('display.max_rows', None)
pd.isnull(stats).sum()

Player     0
Pos        0
Age        0
Tm         0
G          0
GS         0
MP         0
FG         0
FGA        0
FG%        0
3P         0
3PA        0
3P%        0
2P         0
2PA        0
2P%        0
eFG%       0
FT         0
FTA        0
FT%        0
ORB        0
DRB        0
TRB        0
AST        0
STL        0
BLK        0
TOV        0
PF         0
PTS        0
Year       0
Pts Won    0
Pts Max    0
Share      0
Team       0
W          0
L          0
W/L%       0
GB         0
PS/G       0
PA/G       0
SRS        0
T_MP       0
PER        0
TS%        0
3PAr       0
FTr        0
ORB%       0
DRB%       0
TRB%       0
AST%       0
STL%       0
BLK%       0
TOV%       0
USG%       0
OWS        0
DWS        0
WS         0
WS/48      0
OBPM       0
DBPM       0
BPM        0
VORP       0
dtype: int64

In [166]:
stats.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year',
       'Pts Won', 'Pts Max', 'Share', 'Team', 'W', 'L', 'W/L%', 'GB', 'PS/G',
       'PA/G', 'SRS', 'T_MP', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS',
       'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP'],
      dtype='object')

In [167]:
predictors=['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
        '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
        'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year',
        'W', 'L', 'W/L%', 'GB', 'PS/G',
        'PA/G', 'SRS', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%',
        'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS',
        'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP']

In [168]:
train=stats[stats['Year']<2021]
test=stats[stats['Year']==2021]

In [169]:
from sklearn.linear_model import Ridge

reg = Ridge(alpha=.1)

In [170]:
reg.fit(train[predictors], train["Share"])

Ridge(alpha=0.1)

In [171]:
preds=reg.predict(test[predictors])

In [172]:
preds=pd.DataFrame(preds,columns=['predictions'], index=test.index)

In [173]:
pd.set_option('display.max_rows', 15)
preds

,predictions
468,-0.002572
469,-0.004011
470,0.003112
471,-0.025495
472,0.029898
...,...
14087,-0.007080
14088,0.004531
14089,-0.033205
14090,0.115655


In [174]:
combo=pd.concat([test[["Player","Share"]], preds], axis=1)

In [175]:
combo

,Player,Share,predictions
468,Bogdan Bogdanović,0.000,-0.002572
469,Brandon Goodwin,0.000,-0.004011
470,Bruno Fernando,0.000,0.003112
471,Cam Reddish,0.000,-0.025495
472,Clint Capela,0.000,0.029898
...,...,...,...
14087,Raul Neto,0.000,-0.007080
14088,Robin Lopez,0.000,0.004531
14089,Rui Hachimura,0.000,-0.033205
14090,Russell Westbrook,0.005,0.115655


In [176]:
combo.sort_values("Share", ascending=False).head(10)

,Player,Share,predictions
3514,Nikola Jokić,0.961,0.198743
10473,Joel Embiid,0.580,0.084526
4473,Stephen Curry,0.449,0.103980
7690,Giannis Antetokounmpo,0.345,0.185846
10935,Chris Paul,0.138,0.056352
3032,Luka Dončić,0.042,0.148779
11400,Damian Lillard,0.038,0.090781
9290,Julius Randle,0.020,0.095244
9285,Derrick Rose,0.010,0.009928
13500,Rudy Gobert,0.008,0.093230


In [177]:
from sklearn.metrics import mean_squared_error

mean_squared_error(combo["Share"],combo['predictions'])

0.0022840717056302182

In [178]:
combo['Share'].value_counts()

Share
0.000    525
0.001      3
0.042      1
0.961      1
0.449      1
0.345      1
0.010      1
0.020      1
0.003      1
0.580      1
0.138      1
0.038      1
0.008      1
0.005      1
Name: count, dtype: int64

In [179]:
combo=combo.sort_values('Share',ascending=False)
combo['Rk']=list(range(1,combo.shape[0]+1))

In [180]:
combo.head(10)

,Player,Share,predictions,Rk
3514,Nikola Jokić,0.961,0.198743,1
10473,Joel Embiid,0.580,0.084526,2
4473,Stephen Curry,0.449,0.103980,3
7690,Giannis Antetokounmpo,0.345,0.185846,4
10935,Chris Paul,0.138,0.056352,5
3032,Luka Dončić,0.042,0.148779,6
11400,Damian Lillard,0.038,0.090781,7
9290,Julius Randle,0.020,0.095244,8
9285,Derrick Rose,0.010,0.009928,9
13500,Rudy Gobert,0.008,0.093230,10


In [181]:
combo=combo.sort_values('predictions', ascending=False)
combo['Predicted_Rk']=list(range(1,combo.shape[0]+1))

In [182]:
combo.head(10)

,Player,Share,predictions,Rk,Predicted_Rk
3514,Nikola Jokić,0.961,0.198743,1,1
7690,Giannis Antetokounmpo,0.345,0.185846,4,2
3032,Luka Dončić,0.042,0.148779,6,3
14090,Russell Westbrook,0.005,0.115655,11,4
4473,Stephen Curry,0.449,0.103980,3,5
8799,Zion Williamson,0.000,0.103767,73,6
6400,LeBron James,0.001,0.102669,15,7
9290,Julius Randle,0.020,0.095244,8,8
13500,Rudy Gobert,0.008,0.093230,10,9
11400,Damian Lillard,0.038,0.090781,7,10


In [183]:
combo.sort_values("Share",ascending=False).head(10)

,Player,Share,predictions,Rk,Predicted_Rk
3514,Nikola Jokić,0.961,0.198743,1,1
10473,Joel Embiid,0.580,0.084526,2,11
4473,Stephen Curry,0.449,0.103980,3,5
7690,Giannis Antetokounmpo,0.345,0.185846,4,2
10935,Chris Paul,0.138,0.056352,5,20
3032,Luka Dončić,0.042,0.148779,6,3
11400,Damian Lillard,0.038,0.090781,7,10
9290,Julius Randle,0.020,0.095244,8,8
9285,Derrick Rose,0.010,0.009928,9,121
13500,Rudy Gobert,0.008,0.093230,10,9


In [184]:
def find_ap(combo:pd.DataFrame):
    actual=combo.sort_values("Share", ascending=False).head(5)
    predicted=combo.sort_values("predictions", ascending=False)
    ps=[]
    found=0
    seen=1
    for i,r in predicted.iterrows():
        if r['Player'] in actual['Player'].values:
            found+=1
            ps.append(found/seen)
        seen+=1
        
    return sum(ps)/len(ps)

In [185]:
find_ap(combo=combo)

0.6427272727272728

In [186]:
predictors

['Age',
 'G',
 'GS',
 'MP',
 'FG',
 'FGA',
 'FG%',
 '3P',
 '3PA',
 '3P%',
 '2P',
 '2PA',
 '2P%',
 'eFG%',
 'FT',
 'FTA',
 'FT%',
 'ORB',
 'DRB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 'Year',
 'W',
 'L',
 'W/L%',
 'GB',
 'PS/G',
 'PA/G',
 'SRS',
 'PER',
 'TS%',
 '3PAr',
 'FTr',
 'ORB%',
 'DRB%',
 'TRB%',
 'AST%',
 'STL%',
 'BLK%',
 'TOV%',
 'USG%',
 'OWS',
 'DWS',
 'WS',
 'WS/48',
 'OBPM',
 'DBPM',
 'BPM',
 'VORP']

In [187]:
years=list(range(1991,2022))

In [188]:
aps=[]
all_preds=[]
for year in years[5:]:
    train=stats[stats["Year"]<year]
    test=stats[stats["Year"]==year]
    reg.fit(train[predictors],train["Share"])
    preds=reg.predict(test[predictors])
    preds=pd.DataFrame(preds,columns=['predictions'],index=test.index)
    combo=pd.concat([test[["Player","Share"]],preds],axis=1)
    all_preds.append(combo)
    aps.append(find_ap(combo))


In [189]:
sum(aps)/len(aps)

0.7734784583128149

In [190]:
def add_ranks(combo:pd.DataFrame)->pd.DataFrame:
    combo=combo.sort_values('Share',ascending=False)
    combo['Rk']=list(range(1,combo.shape[0]+1))
    combo=combo.sort_values('predictions', ascending=False)
    combo['Predicted_Rk']=list(range(1,combo.shape[0]+1))
    combo['Diff']=combo["Rk"]-combo['Predicted_Rk']
    return combo

In [191]:
ranking=add_ranks(all_preds[1])
ranking[ranking['Rk']<6].sort_values('Diff', ascending=False)

,Player,Share,predictions,Rk,Predicted_Rk,Diff
1670,Michael Jordan,0.832,0.254972,2,1,1
3610,Grant Hill,0.327,0.218286,3,3,0
13135,Karl Malone,0.857,0.247365,1,2,-1
6833,Tim Hardaway,0.207,0.122020,4,8,-4
1377,Glen Rice,0.117,0.025898,5,64,-59


In [192]:
def backtest(stats, model, year, predictors):
    aps=[]
    all_preds=[]
    for year in years[5:]:
        train=stats[stats["Year"]<year]
        test=stats[stats["Year"]==year]
        model.fit(train[predictors],train["Share"])
        preds=reg.predict(test[predictors])
        preds=pd.DataFrame(preds,columns=['predictions'],index=test.index)
        combo=pd.concat([test[["Player","Share"]],preds],axis=1)
        combo=add_ranks(combo)
        all_preds.append(combo)
        aps.append(find_ap(combo))
    return sum(aps)/len(aps),aps,pd.concat(all_preds)

In [193]:
mean_ap,aps,all_preds=backtest(stats,reg, years[5:], predictors)

In [194]:
mean_ap

0.7734784583128149

In [195]:
all_preds[all_preds['Rk']<=5].sort_values("Diff").head(10)

,Player,Share,predictions,Rk,Predicted_Rk,Diff
1377,Glen Rice,0.117,0.025898,5,64,-59
10605,Jason Kidd,0.135,0.016493,5,63,-58
10697,Steve Nash,0.839,0.033186,1,53,-52
11606,Peja Stojaković,0.228,0.031658,4,52,-48
10715,Steve Nash,0.739,0.051923,1,38,-37
8345,Jason Kidd,0.712,0.048377,2,33,-31
10730,Steve Nash,0.785,0.074515,2,21,-19
3744,Chauncey Billups,0.344,0.079492,5,22,-17
12212,Tony Parker,0.274,0.059673,5,21,-16
4439,Stephen Curry,0.173,0.073753,5,21,-16


In [196]:
#compare look at stats of player with large diff

In [197]:
pd.set_option('display.max_rows', None)
pd.concat([pd.Series(reg.coef_), pd.Series(predictors)], axis=1).sort_values(0, ascending=False)

,0,1
13,0.124138,eFG%
29,0.068912,W/L%
18,0.034495,DRB
49,0.028072,WS/48
15,0.025862,FTA
53,0.023016,VORP
16,0.021588,FT%
5,0.018568,FGA
23,0.016105,TOV
17,0.014277,ORB


In [198]:
stat_ratios=stats[["PTS","AST","STL","BLK","3P","WS","VORP","BPM",'PER',"Year"]].groupby("Year").apply(lambda x: x/x.mean())

C:\Users\zac15\AppData\Local\Temp\ipykernel_27328\2288081948.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stat_ratios=stats[["PTS","AST","STL","BLK","3P","WS","VORP","BPM",'PER',"Year"]].groupby("Year").apply(lambda x: x/x.mean())


In [199]:
stat_ratios.index=stat_ratios.index.droplevel()

In [200]:
pd.set_option('display.max_rows', 20)
stat_ratios

,PTS,AST,STL,BLK,3P,WS,VORP,BPM,PER,Year
0,1.692601,2.010078,2.608773,1.346939,5.594452,2.166399,3.728119,-1.296067,1.220476,1.0
1,2.884104,1.542618,2.059558,1.795918,5.085865,3.920150,7.743017,-3.342488,1.770444,1.0
2,0.679268,0.327222,0.549215,0.673469,0.000000,0.653358,-0.143389,1.637137,0.934192,1.0
3,0.055678,0.000000,0.000000,0.673469,0.000000,0.000000,0.000000,3.547130,0.655441,1.0
4,1.514433,1.262142,0.823823,0.224490,1.017173,1.375491,0.716946,0.613926,1.235544,1.0
...,...,...,...,...,...,...,...,...,...,...
14087,0.972892,1.157179,1.780576,0.240964,0.997414,1.373921,1.025836,0.343402,1.011440,1.0
14088,1.006440,0.402497,0.323741,1.445783,0.099741,1.815538,0.820669,0.572337,1.299311,1.0
14089,1.543209,0.704370,1.294964,0.240964,0.797931,1.030441,-1.025836,1.774245,0.886955,1.0
14090,2.482553,5.886518,2.266187,0.963855,1.296638,1.815538,6.975684,-2.117647,1.517160,1.0


In [ ]:
#stats[["PTS_R","AST_R","STL_R","BLK_R","3P_R","VORP_R"]]=stat_ratios[["PTS","AST","STL","BLK","3P","VORP"]]

In [ ]:
#stats

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,OBPM,DBPM,BPM,VORP,PTS_R,AST_R,STL_R,BLK_R,3P_R,VORP_R
0,Doc Rivers,PG,29,ATL,79,79,32.7,5.6,12.9,0.435,...,1.0,1.0,1.9,2.6,1.692601,2.010078,2.608773,1.346939,5.594452,3.728119
1,Dominique Wilkins,SF,31,ATL,81,81,38.0,9.5,20.2,0.470,...,5.4,-0.5,4.9,5.4,2.884104,1.542618,2.059558,1.795918,5.085865,7.743017
2,Duane Ferrell,SF,25,ATL,78,2,14.9,2.2,4.6,0.489,...,-1.3,-1.1,-2.4,-0.1,0.679268,0.327222,0.549215,0.673469,0.000000,-0.143389
3,Gary Leonard,C,23,ATL,4,0,2.3,0.0,0.0,0.000,...,-6.3,1.1,-5.2,0.0,0.055678,0.000000,0.000000,0.673469,0.000000,0.000000
4,John Battle,SG,28,ATL,79,2,23.6,5.0,10.9,0.461,...,0.7,-1.6,-0.9,0.5,1.514433,1.262142,0.823823,0.224490,1.017173,0.716946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14087,Raul Neto,PG,28,WAS,64,22,21.9,3.3,7.0,0.468,...,-1.4,0.9,-0.6,0.5,0.972892,1.157179,1.780576,0.240964,0.997414,1.025836
14088,Robin Lopez,C,32,WAS,71,9,19.1,3.8,6.0,0.633,...,0.2,-1.2,-1.0,0.4,1.006440,0.402497,0.323741,1.445783,0.099741,0.820669
14089,Rui Hachimura,PF,22,WAS,57,57,31.5,5.4,11.4,0.478,...,-2.2,-1.0,-3.1,-0.5,1.543209,0.704370,1.294964,0.240964,0.797931,-1.025836
14090,Russell Westbrook,PG,32,WAS,65,65,36.4,8.4,19.0,0.439,...,2.6,1.1,3.7,3.4,2.482553,5.886518,2.266187,0.963855,1.296638,6.975684


In [ ]:
test_predictors=['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
        '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
        'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year',
        'W', 'L', 'W/L%', 'GB', 'PS/G',
        'PA/G', 'SRS', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%',
        'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS',
        'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP']

In [204]:
mean_ap,aps,all_preds=backtest(stats,reg, years[5:], predictors)

In [205]:
mean_ap

0.7734784583128149

In [206]:
stats["NPos"]=stats["Pos"].astype("category").cat.codes
stats["NTm"]=stats["Tm"].astype("category").cat.codes

In [ ]:
#Random Forest
from sklearn.ensemble import RandomForestRegressor

rf=RandomForestRegressor(n_estimators=500, random_state=15, min_samples_split=7,n_jobs=-1)

mean_ap,aps,all_preds=backtest(stats,rf, years[5:], predictors)